Author: Santhilata KV

**This code sheet is to create folder structure on windows. Transfering data from objective to x_drive**

In [ ]:
import pandas as pd
import numpy as np

import os
import string
import re

import csv

In [ ]:
#Datasheet_path = '/home/santhilata/Dropbox/AI_for_Selection/Code/X_drive_categories_paul.csv'
#objective_data_sheet_path = '/home/santhilata/Dropbox/AI_for_Selection/Code/Copy of Prepare folders docs for research_DATE.xlsx'
objective_data_sheet_path = input('Input data sheet : ')

In [ ]:
obj_df = pd.read_excel(open(objective_data_sheet_path,'rb'),sheet_name='data')

In [ ]:
obj_df.columns

In [ ]:
obj_df = obj_df.drop(['trimparent11', 'trimparent10',
       'trimparent9', 'trimparent8', 'trimparent7', 'trimparent6',
       'trimparent5', 'trimparent4', 'trimparent3', 'trimparent2',
       'trimparent1', 'trimdocumentname', 'fullpathcalculated',
       'fullpathcalculatedincdocument', 'pathlength'], axis=1)

In [ ]:
obj_df.columns

In [ ]:
obj_df = obj_df.fillna(value='no name', axis=0)

In [ ]:
folder_dict = {} # create a dictionary to store real and trimmed folder names 

#Actually the following is not necessary
def check_splcharacter(ch): 
  
    # Make an RE character set and pass  
    # this as an argument in compile function
 
    string_check= re.compile('[@_!#$%^&*()<>?/\|}{~:]') 
      
    # Pass the string in search function  
    # of RE object (string_check).
     
    if(string_check.search(ch) == None): 
        return False
          
    else: 
        return True

#print(check_splcharacter('1'))


# function to get the trimmed name from the original folder name
def trim_fn_name_space(folderName):
     
    acronym = ''
    if folderName not in folder_dict.keys():
        
        print(folderName,' ',)
        if folderName =='no name':
            acronym = 'no name'
            
        else:
            if (check_splcharacter(folderName[0])):
                acronym = '_'
            else:
                acronym = folderName[0]
            
            char_counter = 1
            for ch in folderName[1:]:
                
                    
                if (ch == ' ') & (char_counter < len(folderName)) :
                    if (folderName[char_counter+1] in string.ascii_lowercase)  :
                        if (check_splcharacter(folderName[char_counter+1])):
                            acronym = acronym+ '_'
                        else:
                            acronym = acronym+folderName[char_counter+1]
                    
                if (ch in string.ascii_uppercase) |(ch.isdigit() )  :
                    if (check_splcharacter(ch)):
                        acronym = acronym+'_'
                    else:
                        acronym = acronym+str(ch) 
                char_counter += 1
       
        #The following is to take care of identical acronyms. A number is appended
        #Assumption: No more than 99 files may have same acronyms
        while (acronym  in folder_dict.values()):
            
            if (acronym[-1] in string.digits) and (acronym[-2] == '_'):
                new_val = int(acronym[-1])+1
                acronym = acronym[:-1]+str(new_val)
        
            elif (acronym[-1] in string.digits)and (acronym[-2] in string.digits) and (acronym[-3] == '_'):
                
                new_val = 10*int(acronym[-2])+int(acronym[-1])+1
                acronym = acronym[:-2]+str(new_val)
            else:
                acronym = acronym+'_1'
    
        print(acronym)        
        folder_dict[folderName] = acronym
        
    return folder_dict[folderName]

In [ ]:
# now create new trimmed columns

for i in range(11,0,-1):
    
    obj_df['trim_'+str(i)] = obj_df['parent'+str(i)].apply(lambda x: trim_fn_name_space(str(x)))

In [ ]:
# Write all acronyms from dictionary to csv
import csv
csv_columns = ['Folder name','Acronym']
csv_file = "folders_acronyms.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for key in folder_dict.keys():
            writer.writerow({'Folder name': key,'Acronym':folder_dict[key]})
except IOError:
    print("I/O error") 

In [ ]:
#Write obj_df to csv
obj_df.to_csv('objective_files_with_acronyms.csv',sep=',')

In [ ]:
obj_df.columns

In [ ]:
#test assert
s = set( val for val in folder_dict.values())
print(len(s))